In [1]:
!apt install wget
!pip install bert-tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
#tf.enable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
with open('data/counts_1grams.txt') as fopen:
    f = fopen.read().split('\n')[:-1]
    
words = {}
for l in f:
    w, c = l.split('\t')
    c = int(c)
    words[w] = c + words.get(w, 0)


In [4]:
list(words.items())[:10]
words["ewing"]

1

In [5]:
# original from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/classes/spellcorrect.py
# improved it

import re
from collections import Counter

class SpellCorrector:
    """
    The SpellCorrector extends the functionality of the Peter Norvig's
    spell-corrector in http://norvig.com/spell-correct.html
    """
    REGEX_TOKEN = re.compile(r'\b[a-z]{2,}\b')

    def __init__(self, words):
        """
        :param corpus: the statistics from which corpus to use for the spell correction.
        """
        super().__init__()
        self.WORDS = words
        self.N = sum(self.WORDS.values())
              
    @staticmethod
    def tokens(text):
        return REGEX_TOKEN.findall(text.lower())

    def P(self, word):
        """
        Probability of `word`.
        """
        return self.WORDS[word] / self.N

    def most_probable(self, words):
        _known = self.known(words)
        if _known:
            return max(_known, key=self.P)
        else:
            return []

    @staticmethod
    def edit_step(word):
        """
        All edits that are one edit away from `word`.
        """
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        All edits that are two edits away from `word`.
        """
        return (e2 for e1 in self.edit_step(word)
                for e2 in self.edit_step(e1))

    def known(self, words):
        """
        The subset of `words` that appear in the dictionary of WORDS.
        """
        return set(w for w in words if w in self.WORDS)

    def edit_candidates(self, word, assume_wrong=False, fast=True):
        """
        Generate possible spelling corrections for word.
        """

        if fast:
            ttt = self.known(self.edit_step(word)) or {word}
        else:
            ttt = self.known(self.edit_step(word)) or self.known(self.edits2(word)) or {word}
        
        ttt = self.known([word]) | ttt
        return list(ttt)

In [6]:
corrector = SpellCorrector(words)

In [7]:
#possible_states = corrector.edit_candidates('eting')
possible_states = corrector.edit_candidates('gife')
possible_states

['gibe', 'wife', 'life', 'rife', 'gift', 'give']

In [8]:
#!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [9]:
    
#!unzip uncased_L-12_H-768_A-12.zip

In [10]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [11]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np

W0128 18:07:28.686888 139848485451584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

W0128 18:07:28.794579 139848485451584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [13]:
#text = 'scientist suggests eting burger can lead to obesity'
text = 'gife me something to eat'
#text_mask = text.replace('eting', '**mask**')
text_mask = text.replace('gife', '**mask**')
text_mask

'**mask** me something to eat'

In [14]:
def tokens_to_masked_ids(tokens, mask_ind):
    masked_tokens = tokens[:]
    masked_tokens[mask_ind] = "[MASK]"
    masked_tokens = ["[CLS]"] + masked_tokens + ["[SEP]"]
    masked_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
    return masked_ids

In [15]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [16]:
bert_config

In [17]:
class Model:
    def __init__(self,):
        self.X = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        #print(modeling.get_activation(bert_config.hidden_act))
        print(output_layer.shape)
        print(embedding.shape)
        print(bert_config.hidden_size)
        print(bert_config.initializer_range)
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
                print(input_tensor.shape)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [18]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')

W0128 18:07:29.625540 139848485451584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0128 18:07:29.628924 139848485451584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0128 18:07:29.673655 139848485451584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0128 18:07:30.457144 139848485451584 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (

(?, ?, 768)
(30522, 768)
768
0.02
(?, ?, 768)


In [19]:
var_lists

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(30522, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bia

In [20]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [21]:
saver = tf.train.Saver(var_list = var_lists + cls)
#saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)

W0128 18:07:37.368971 139848485451584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [22]:
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [23]:
replaced_masks = [text_mask.replace('**mask**', state) for state in possible_states]
replaced_masks

['gibe me something to eat',
 'wife me something to eat',
 'life me something to eat',
 'rife me something to eat',
 'gift me something to eat',
 'give me something to eat']

In [24]:
def get_score(mask):
    tokens = tokenizer.tokenize(mask)
    print(tokens)
    input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
    print(input_ids)
    preds = sess.run(tf.nn.softmax(model.logits), feed_dict = {model.X: input_ids})
    print(preds.shape)
    #print(preds)
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    print([preds[i, i + 1, x] for i, x in enumerate(tokens_ids)])
    return np.prod([preds[i, i + 1, x] for i, x in enumerate(tokens_ids)])

In [25]:
get_score(replaced_masks[0])

['gi', '##be', 'me', 'something', 'to', 'eat']
[[101, 103, 4783, 2033, 2242, 2000, 4521, 102], [101, 21025, 103, 2033, 2242, 2000, 4521, 102], [101, 21025, 4783, 103, 2242, 2000, 4521, 102], [101, 21025, 4783, 2033, 103, 2000, 4521, 102], [101, 21025, 4783, 2033, 2242, 103, 4521, 102], [101, 21025, 4783, 2033, 2242, 2000, 103, 102]]
(6, 8, 30522)
[0.0002885414, 5.6245067e-06, 7.2859875e-05, 0.020616144, 0.8899409, 5.954206e-06]


1.291735e-20

In [26]:
get_score(replaced_masks[1])

['wife', 'me', 'something', 'to', 'eat']
[[101, 103, 2033, 2242, 2000, 4521, 102], [101, 2564, 103, 2242, 2000, 4521, 102], [101, 2564, 2033, 103, 2000, 4521, 102], [101, 2564, 2033, 2242, 103, 4521, 102], [101, 2564, 2033, 2242, 2000, 103, 102]]
(5, 7, 30522)
[9.35459e-07, 0.00036564094, 0.0012138827, 0.74889284, 7.788646e-06]


2.4217983e-18

In [27]:
get_score(replaced_masks[4])

['gift', 'me', 'something', 'to', 'eat']
[[101, 103, 2033, 2242, 2000, 4521, 102], [101, 5592, 103, 2242, 2000, 4521, 102], [101, 5592, 2033, 103, 2000, 4521, 102], [101, 5592, 2033, 2242, 103, 4521, 102], [101, 5592, 2033, 2242, 2000, 103, 102]]
(5, 7, 30522)
[5.1484534e-05, 0.43783578, 0.9418497, 0.9781953, 7.0758088e-06]


1.4695058e-10

In [28]:
get_score('me give something to eat')

['me', 'give', 'something', 'to', 'eat']
[[101, 103, 2507, 2242, 2000, 4521, 102], [101, 2033, 103, 2242, 2000, 4521, 102], [101, 2033, 2507, 103, 2000, 4521, 102], [101, 2033, 2507, 2242, 103, 4521, 102], [101, 2033, 2507, 2242, 2000, 103, 102]]
(5, 7, 30522)
[0.00016578873, 6.711345e-05, 0.07836451, 0.44484338, 9.913343e-07]


3.8451317e-16

In [29]:
scores = [get_score(mask) for mask in replaced_masks]
scores

['gi', '##be', 'me', 'something', 'to', 'eat']
[[101, 103, 4783, 2033, 2242, 2000, 4521, 102], [101, 21025, 103, 2033, 2242, 2000, 4521, 102], [101, 21025, 4783, 103, 2242, 2000, 4521, 102], [101, 21025, 4783, 2033, 103, 2000, 4521, 102], [101, 21025, 4783, 2033, 2242, 103, 4521, 102], [101, 21025, 4783, 2033, 2242, 2000, 103, 102]]
(6, 8, 30522)
[0.0002885414, 5.6245067e-06, 7.2859875e-05, 0.020616144, 0.8899409, 5.954206e-06]
['wife', 'me', 'something', 'to', 'eat']
[[101, 103, 2033, 2242, 2000, 4521, 102], [101, 2564, 103, 2242, 2000, 4521, 102], [101, 2564, 2033, 103, 2000, 4521, 102], [101, 2564, 2033, 2242, 103, 4521, 102], [101, 2564, 2033, 2242, 2000, 103, 102]]
(5, 7, 30522)
[9.35459e-07, 0.00036564094, 0.0012138827, 0.74889284, 7.788646e-06]
['life', 'me', 'something', 'to', 'eat']
[[101, 103, 2033, 2242, 2000, 4521, 102], [101, 2166, 103, 2242, 2000, 4521, 102], [101, 2166, 2033, 103, 2000, 4521, 102], [101, 2166, 2033, 2242, 103, 4521, 102], [101, 2166, 2033, 2242, 2000, 10

[1.291735e-20,
 2.4217983e-18,
 1.7506237e-19,
 2.3278124e-18,
 1.4695058e-10,
 1.3814047e-06]

In [30]:
print(np.sum(scores))
print(np.argmax(scores))
prob_scores = np.array(scores) / np.sum(scores)
print(prob_scores)
print(np.argmax(prob_scores))
print(possible_states)

1.3815517e-06
5
[9.3498859e-15 1.7529553e-12 1.2671431e-13 1.6849261e-12 1.0636633e-04
 9.9989361e-01]
5
['gibe', 'wife', 'life', 'rife', 'gift', 'give']


In [31]:
list(zip(possible_states, prob_scores))

[('gibe', 9.349886e-15),
 ('wife', 1.7529553e-12),
 ('life', 1.2671431e-13),
 ('rife', 1.6849261e-12),
 ('gift', 0.00010636633),
 ('give', 0.9998936)]